# Getting Your Data From Yelp!

In order to make sure you are on track to completing the project, you will complete this workbook first. Below are steps that you need to take in order to make sure you have your data from yelp and are ready to analyze it. Your cohort lead will review this workbook with you the Wednesday before your project is due.    

# Part 1 - Understanding your data and question

You will be pulling data from the Yelp API to complete your analysis. The API, however, provides you with a lot of information that will not be pertinent to your analysis. You will pull data from the API and parse through it to keep only the data that you will need. In order to help you identify that information,look at the API documentation and understand what data the API will provide you. 

Identify which data fields you will want to keep for your analysis. 

https://www.yelp.com/developers/documentation/v3/get_started

# Part 2 - Create ETL pipeline for the business data from the API

## Details

Now that you know what data you need from the API, you want to write code that will execute an API call, parse those results and then insert the results into the DB.  

It is helpful to break this up into three different functions (*API call, parse results, and insert into DB*) and then you can write a function/script that pull the other three functions together. 

Let's first do this for the Business endpoint.

## Request

### Imports and Setup

In [1]:
import requests
import pandas as pd
import json
import csv

with open(r'C:\Users\bmcca\.secret\yelp_api.json') as f:
    keys = json.load(f)

client_id = keys['id']
yelp_key = keys['key']

### ƒ: yelp_request

 - Params: search term (eg. "wineries); location; yelp_key variable (from Imports); and changing setting to print details

In [2]:
def yelp_request(term, location, yelp_key, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50
                }
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

### Sending the request and saving the response

- Un-comment the next line to run the response

In [3]:
response = yelp_request('winery','Southern California', yelp_key)
response.keys()

<Response [200]>
<class 'str'>
{"businesses": [{"id": "4N2pC-h3Ba6qUQ7cN4xP-g", "alias": "san-antonio-winery-ontario", "name": "San Antonio Winery", "image_url": "https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw", "review_count": 248, "categories": [{"alias": "wineries", "title": "Wineries"}], "rating": 4.0, "coordinates": {"latitude": 34.022224, "longitude": -117.558985}, "transactions": [], "price": "$$", "location": {"address1": "2802 S Milliken Ave", "address2": null, "address3": "", "city": "Ontario", "zip_code": "91761", "country": "US", "state": "CA", "display_address": ["2802 S Milliken Ave", "Ontario, CA 91761"]}, "phone": "+19099473995", "display_phone": "(909) 947-3995", "distance": 26348.268371330654}, {"id": "Q0Cuh5XZY29AdrWpUv8yhw", "alias": 

dict_keys(['businesses', 'total', 'region'])

#### Saving/Loading as JSON for simplicity while iterating

In [4]:
# try:
#     with open(r'data\response.txt', 'w') as f:
#         json.dump(response, f)
# except IOError:
#     print("I/O error")

In [5]:
# with open('data/response.txt') as json_file:
#     data = json.load(json_file)

### Identifying and Exploring Keys

In [6]:
# Identify keys

print(response.keys())

dict_keys(['businesses', 'total', 'region'])


#### Exploring the "Businesses" Key

In [7]:
response['businesses']

[{'id': '4N2pC-h3Ba6qUQ7cN4xP-g',
  'alias': 'san-antonio-winery-ontario',
  'name': 'San Antonio Winery',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
  'review_count': 248,
  'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
  'rating': 4.0,
  'coordinates': {'latitude': 34.022224, 'longitude': -117.558985},
  'transactions': [],
  'price': '$$',
  'location': {'address1': '2802 S Milliken Ave',
   'address2': None,
   'address3': '',
   'city': 'Ontario',
   'zip_code': '91761',
   'country': 'US',
   'state': 'CA',
   'display_address': ['2802 S Milliken Ave', 'Ontario, CA 91761']},
  'phone': '+19099473995',
  'display_phone': '(909) 947-3995',
  'distance': 26348.268371330654},
 {'id': 'Q0Cuh5XZY29AdrWpUv8yhw',
  '

In [8]:
# Show first item w/in list of businesses

response['businesses'][0]

{'id': '4N2pC-h3Ba6qUQ7cN4xP-g',
 'alias': 'san-antonio-winery-ontario',
 'name': 'San Antonio Winery',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/__5IBirgkaHoRGdVnUIdng/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/san-antonio-winery-ontario?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
 'review_count': 248,
 'categories': [{'alias': 'wineries', 'title': 'Wineries'}],
 'rating': 4.0,
 'coordinates': {'latitude': 34.022224, 'longitude': -117.558985},
 'transactions': [],
 'price': '$$',
 'location': {'address1': '2802 S Milliken Ave',
  'address2': None,
  'address3': '',
  'city': 'Ontario',
  'zip_code': '91761',
  'country': 'US',
  'state': 'CA',
  'display_address': ['2802 S Milliken Ave', 'Ontario, CA 91761']},
 'phone': '+19099473995',
 'display_phone': '(909) 947-3995',
 'distance': 26348.268371330654}

In [70]:
response['businesses'][0]['categories'][0]['alias']

'wineries'

In [69]:
response['businesses'][0]['categories'][0]['title']

'Wineries'

#### Exploring the "Total" Key

In [9]:
response['total']

# How many reviews are there in total for my request?

491

#### Exploring the "Region" Key

In [10]:
response['region']

# From which geographical area will my results come?

{'center': {'longitude': -117.784423828125, 'latitude': 34.16815524147703}}

## Parse

In [12]:
response.keys()

dict_keys(['businesses', 'total', 'region'])

### ƒ: parse_data

In [73]:
def parse_data(list_of_data):
    '''Adapted from Tyrell's code'''  


    parsed_data = []
     
    for business in list_of_data:
        if 'price' in business:
            details = {'name': business['name'],
                         'location': business['location']['display_address'],
                         'id': business['id'],
                         #'categories': business['categories'],
                         'alias': business['categories'][0]['alias'],
                         'title': business['categories'][0]['title'],
                         'rating': business['rating'],
                         'review_count': business['review_count'],
                         'price': business['price'],
                         'latitude': business['coordinates']['latitude'],
                         'longitude': business['coordinates']['longitude']
                        }
        else:
            details = {'name': business['name'],
                         'location': business['location']['display_address'],
                         'id': business['id'],
                         #'categories': business['categories'],
                         'alias': business['categories'][0]['alias'],
                         'title': business['categories'][0]['title'],
                         'rating': business['rating'],
                         'review_count': business['review_count'],
                         'latitude': business['coordinates']['latitude'],
                         'longitude': business['coordinates']['longitude']
                        }
    
        parsed_data.append(details)
    
    for biz in parsed_data:
        biz['location'] = ' '.join(biz['location'])
        
    df_parsed_data = pd.DataFrame(parsed_data)
    
#     df_parsed_data.dropna(inplace=True)
    
    return df_parsed_data

In [74]:
parsed_results = parse_data(response['businesses'])
parsed_results

,name,location,id,alias,title,rating,review_count,price,latitude,longitude
0,San Antonio Winery,"2802 S Milliken Ave Ontario, CA 91761",4N2pC-h3Ba6qUQ7cN4xP-g,wineries,Wineries,4.0,248,$$,34.022224,-117.558985
1,Joseph Filippi Winery,"12467 Baseline Rd Rancho Cucamonga, CA 91739",Q0Cuh5XZY29AdrWpUv8yhw,wineries,Wineries,4.0,158,$$,34.120926,-117.533615
2,Third Street Wine Shop,"2142 3rd St La Verne, CA 91750",Liy2V5TissVcWFt9-45AKg,wine_bars,Wine Bars,5.0,101,$$,34.099990,-117.769460
3,Galleano Winery,"4231 Wineville Ave Mira Loma, CA 91752",dMy0GL2mG9hKIIMsfAQAgQ,winetastingroom,Wine Tasting Room,4.0,171,$$,34.010900,-117.542460
4,The Claremont Packing House,"532 W 1st St Claremont, CA 91711",jJYkAV0acTPOkqESRWjmTA,winetastingroom,Wine Tasting Room,4.0,23,$$,34.094265,-117.721303
5,San Dimas Wine Shop & Tasting Room,"225 W Bonita Ave San Dimas, CA 91773",wXps6c1vd4iu__NO3fV3Rg,beer_and_wine,"Beer, Wine & Spirits",4.5,82,$$,34.106990,-117.810290
6,Hartlam Winery,"864 Commercial Ave San Gabriel, CA 91776",HDHWYVEIhcs039jfkIIuKg,wineries,Wineries,5.0,1,NaN,34.095950,-118.088710
7,Winery At Main Street,"4861 Main St Yorba Linda, CA 92886",06kNBRr19DOvyB83mfBjVA,wineries,Wineries,4.5,112,$$,33.890499,-117.814270
8,San Antonio Winery,"737 Lamar St Los Angeles, CA 90031",HlKxlisBOQ80uO6vcetlBA,wineries,Wineries,4.0,1392,$$,34.063686,-118.223848
9,Sycamore Ranch Vineyard & Winery,"Dart Canyon Crestline, CA 92325",CFpBn9x8m4r2tPh7WGevww,winetastingroom,Wine Tasting Room,5.0,33,$$,34.258615,-117.264943


In [75]:
parsed_results.isnull().sum()

name             0
location         0
id               0
alias            0
title            0
rating           0
review_count     0
price           13
latitude         0
longitude        0
dtype: int64

In [16]:
parsed_results.describe()

,rating,review_count,latitude,longitude
count,50.000000,50.00000,50.000000,50.000000
mean,4.430000,116.62000,34.059168,-117.875246
std,0.505177,225.83766,0.301665,0.346680
min,3.000000,1.00000,33.559875,-118.687958
25%,4.000000,8.25000,33.816319,-118.133451
50%,4.500000,48.00000,34.060279,-117.900933
75%,5.000000,118.75000,34.149483,-117.677867
max,5.000000,1392.00000,34.696100,-117.039049


## Updating Requests for Pagination

### ƒ: yelp_request_offset

In [17]:
def yelp_request_offset(term, location, yelp_key, offset=0, verbose=True):
    '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
            'Authorization': 'Bearer {}'.format(yelp_key),
        }

    url_params = {
                    'term': term.replace(' ', '+'),
                    'location': location.replace(' ', '+'),
                    'limit': 50,
                    'offset': offset
                        }
    
    response = requests.get(url, headers=headers, params=url_params)
    
    if verbose == True:
        print(response)
        print(type(response.text))
        print(response.text[:1000])
        
    return response.json()

#### Test 1

In [18]:
# test1 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)
# test1

{'businesses': [{'id': '76ADW8x8J_69qbtsc5F-2g',
   'alias': 'the-winery-restaurant-and-wine-bar-san-diego',
   'name': 'The Winery Restaurant & Wine Bar',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/svRxHx6tJR7I1bUTqp29BA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/the-winery-restaurant-and-wine-bar-san-diego?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
   'review_count': 492,
   'categories': [{'alias': 'bars', 'title': 'Bars'},
    {'alias': 'newamerican', 'title': 'American (New)'}],
   'rating': 4.0,
   'coordinates': {'latitude': 32.8724284, 'longitude': -117.2137748},
   'transactions': ['pickup', 'delivery'],
   'price': '$$',
   'location': {'address1': '4301 La Jolla Village Dr',
    'address2': 'Ste 2040',
    'address3': None,
    'city': 'San Diego',
    'zip_code': '92122',
    'country': 'US',
    'state': 'CA',
    'display_address': ['4301 La Jol

In [19]:
test1.keys()

dict_keys(['businesses', 'total', 'region'])

In [20]:
test1['total']

262

In [21]:
len(test1['businesses'])

50

#### Test 2

In [22]:
# test2 = yelp_request_offset('winery', 'San Diego', yelp_key, offset=50, verbose=False)
# test2

{'businesses': [{'id': 'm4GLKC9LG_aelSqR_BCcEg',
   'alias': 'vino-carta-san-diego',
   'name': 'Vino Carta',
   'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/o93tGRUAsqQh0hMaPupSvQ/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/vino-carta-san-diego?adjust_creative=eyY_X0k6PMB2bu6EM1dXdw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=eyY_X0k6PMB2bu6EM1dXdw',
   'review_count': 63,
   'categories': [{'alias': 'beer_and_wine', 'title': 'Beer, Wine & Spirits'},
    {'alias': 'wine_bars', 'title': 'Wine Bars'}],
   'rating': 4.5,
   'coordinates': {'latitude': 32.727181229033,
    'longitude': -117.169526070356},
   'transactions': ['delivery'],
   'price': '$$',
   'location': {'address1': '2161 India St',
    'address2': '',
    'address3': '',
    'city': 'San Diego',
    'zip_code': '92101',
    'country': 'US',
    'state': 'CA',
    'display_address': ['2161 India St', 'San Diego, CA 92101']},
   'phone': '+16195646589',
   'display_

#### Loop

In [23]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

262
50
6


In [24]:
# cur = 50
# parsed_results_dfs = []

# for num in range(num_pages):
#     results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
#     parsed_results = parse_data(results['businesses'])
#     parsed_results_dfs.append(parsed_results)
#     cur += 50
    

In [76]:
# parsed_results_dfs

In [77]:
# df_concat = pd.concat(parsed_results_dfs, ignore_index=True)
# df_concat

## Save

### ƒ: df_save

Save parsed results to a .csv file

In [36]:
# df_concat.to_csv('data\wineries.csv')

In [38]:
# df = pd.read_csv("data/wineries.csv", index_col=0)
# df

,name,location,id,rating,review_count,price,latitude,longitude
0,Vino Carta,"2161 India St San Diego, CA 92101",m4GLKC9LG_aelSqR_BCcEg,4.5,63,$$,32.727181,-117.169526
1,California Wine Line,"San Diego, CA 92101",5fkdGagUJDuc3axRWchzJQ,5.0,2,NaN,32.605974,-117.044101
2,San Pasqual Winery Tasting Room & Gallery,"8364 La Mesa Blvd La Mesa, CA 91942",ia3GRsrEDexiodnpXe0D6w,4.5,81,$$,32.765720,-117.017870
3,Baja California Wine Tours,"San Diego, CA 92110",-R9x8f3TaNek2JIP-4OHCA,5.0,10,NaN,32.767720,-117.194830
4,The Wine Lover,"3968 5th Ave San Diego, CA 92103",xsnS_WeHv2QYOK2N1PKmTA,4.0,205,$$,32.749550,-117.160780
...,...,...,...,...,...,...,...,...
142,San Diego Limobuses,"3333 Midway Dr Ste 206 San Diego, CA 92110",SCaFGyzrTGTI6aQHhLxbgA,3.0,46,NaN,32.750060,-117.211380
143,Alpine Discount Liquor,"2223 Alpine Blvd Alpine, CA 91901",-ARx5ShNxJgjyahKnikTnA,3.5,3,$,32.835287,-116.765910
144,Village Wine & Spirits,"1552 Encinitas Blvd Encinitas, CA 92024",XkGnb-YxP5MK_ok1X011RA,4.0,24,$$,33.045896,-117.255584
145,The Destination Wedding Group,"Escondido, CA 92033",lJwxe_fjdt-e8xPrDq63fA,5.0,18,NaN,33.123470,-117.086520


#### Old Stuff

In [ ]:
# from helper_funcs import *

In [ ]:


# # create a variable  to keep track of which result you are in. 
# cur = 0

# #set up a while loop to go through and grab the result 
# while cur < num and cur < 1000:
#     #set the offset parameter to be where you currently are in the results 

#     #make your API call with the new offset number
#     results = yelp_call_offset('winery', 'San Diego', yelp_key, offset=cur, verbose=False)
    
#     #after you get your results you can now use your function to parse those results
#     parsed_results = parse_results(results)
    
#     # use your function to insert your parsed results into the db
#     df_save(parsed_results)
#     #increment the counter by 50 to move on to the next results
#     cur += 50

## Edit and Condense

Goal: condense the details down to specific functions (perhaps one function?) to pull, clean, and save data.

While it will take some experimentation to write the functions above, once you get them working it will be best to put them in a `.py` file and then import the functions to use in a script 

### ƒ: simplified_function_name

In [78]:
# results = yelp_request_offset('winery', 'San Diego', yelp_key, offset=0, verbose=False)

# num_pages = results['total']//50+1

# print(results['total'])
# print(len(results['businesses']))
# print(num_pages)

In [48]:
def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
    '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
    # Process first request to Yelp API and calculate number of pages 
    results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
    num_pages = results['total']//50+1
    
    # Print out confirmation feedback
    print(f'For {term} and {location}: ')
    print(f"    Total number of results: {results['total']}.")
    print(f'    Total number of pages: {num_pages}.')
    
    # Create offset for further results and create empty list
    cur = 50
    parsed_results_dfs = []

    # Retrieves remaining pages
    for num in range(num_pages-1):
        try:
            results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
            parsed_results = parse_data(results['businesses'])
            parsed_results_dfs.append(parsed_results)
            cur += 50
        except:
            print(f'Error on page {num}.')

    # Concatenate DataFrames and save to .csv
    df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

    try:
        df_concat.to_csv(file_name)
        print(f'Saved to {file_name}.')
    except:
        print(f'Error, did not save.')
        
    return df_concat

In [49]:
df2 = get_full_data('winery', 'San Diego', yelp_key)
df2

For winery and San Diego: 
    Total number of results: 262.
    Total number of pages: 6.
Saved to data/wineries.csv.


,name,location,id,rating,review_count,price,latitude,longitude
0,Vino Carta,"2161 India St San Diego, CA 92101",m4GLKC9LG_aelSqR_BCcEg,4.5,63,$$,32.727181,-117.169526
1,San Pasqual Winery Tasting Room & Gallery,"8364 La Mesa Blvd La Mesa, CA 91942",ia3GRsrEDexiodnpXe0D6w,4.5,81,$$,32.765720,-117.017870
2,Baja California Wine Tours,"San Diego, CA 92110",-R9x8f3TaNek2JIP-4OHCA,5.0,10,NaN,32.767720,-117.194830
3,The Wine Lover,"3968 5th Ave San Diego, CA 92103",xsnS_WeHv2QYOK2N1PKmTA,4.0,205,$$,32.749550,-117.160780
4,Chuparosa Vineyards,"910 Gem Ln Ramona, CA 92065",D9rtDihupzA39yBh0YcAXg,5.0,39,$$,33.009689,-116.858495
...,...,...,...,...,...,...,...,...
142,San Diego Limobuses,"3333 Midway Dr Ste 206 San Diego, CA 92110",SCaFGyzrTGTI6aQHhLxbgA,3.0,46,NaN,32.750060,-117.211380
143,Alpine Discount Liquor,"2223 Alpine Blvd Alpine, CA 91901",-ARx5ShNxJgjyahKnikTnA,3.5,3,$,32.835287,-116.765910
144,Village Wine & Spirits,"1552 Encinitas Blvd Encinitas, CA 92024",XkGnb-YxP5MK_ok1X011RA,4.0,24,$$,33.045896,-117.255584
145,The Destination Wedding Group,"Escondido, CA 92033",lJwxe_fjdt-e8xPrDq63fA,5.0,18,NaN,33.123470,-117.086520


In [86]:
print(df2['name'].to_string())

0                                         Vino Carta
1          San Pasqual Winery Tasting Room & Gallery
2                         Baja California Wine Tours
3                                     The Wine Lover
4                                Chuparosa Vineyards
5                               California Wine Line
6                         Principe di Tricase Winery
7                       Poochie's Hooch Urban Cidery
8                                   Serpentine Cider
9             Bottlecraft Beer Shop and Tasting Room
10                                Continental Bistro
11                               Aha Baja Wine Tours
12                             Hatfield Creek Winery
13                               Record Family Wines
14                                       Baja Xpress
15                         Dulzura Vineyard & Winery
16                                Quigley Fine Wines
17                                     Wine Smarties
18                                     Cheval 

In [50]:
# df3 = get_full_data('pizza', 'Baltimore', yelp_key)

In [51]:
# df3

## Cleaning Data

In [55]:
df2.isna().sum()

name             0
location         0
id               0
rating           0
review_count     0
price           78
latitude         0
longitude        0
dtype: int64

In [58]:
df2[df2['price'].isna()]

,name,location,id,rating,review_count,price,latitude,longitude
2,Baja California Wine Tours,"San Diego, CA 92110",-R9x8f3TaNek2JIP-4OHCA,5.0,10,NaN,32.767720,-117.194830
5,California Wine Line,"San Diego, CA 92101",5fkdGagUJDuc3axRWchzJQ,5.0,2,NaN,32.605974,-117.044101
7,Poochie's Hooch Urban Cidery,"7559 Mission Gorge Rd San Diego, CA 92120",rbtrzNAu90P2GyyE55kikA,4.0,70,NaN,32.807324,-117.075977
8,Serpentine Cider,"2311 El Cajon Blvd San Diego, CA 92104",JxeaJ07kzO6MYnDJ2rLoeA,4.5,14,NaN,32.755000,-117.139710
10,Continental Bistro,"2539 Congress Street A San Diego, CA 92110",NA9IzEA80BsNBSATIpPDOQ,3.5,3,NaN,32.752220,-117.196385
...,...,...,...,...,...,...,...,...
135,Five Star Tours & Charter Bus Company,"1050 Kettner Blvd San Diego, CA 92101",vA5DTHtZUFx1En8HY_hdRQ,3.0,346,NaN,32.716787,-117.169328
136,5 Star Market & Liquor,"7495 University Ave La Mesa, CA 91941",554hlhnhhj89YpErDfGQdg,4.5,17,NaN,32.758130,-117.036040
138,Turista Libre,"San Diego, CA 92173",L-RzyEzgb6_NSiGOyzL3gA,5.0,169,NaN,32.543800,-117.035790
142,San Diego Limobuses,"3333 Midway Dr Ste 206 San Diego, CA 92110",SCaFGyzrTGTI6aQHhLxbgA,3.0,46,NaN,32.750060,-117.211380


# Part 3 -  Create ETL pipeline for the restaurant review data from the API

You've done this for the Businesses, now you need to do this for reviews. You will follow the same process, but your functions will be specific to reviews. Above you have a model of the functions you will need to write, and how to pull them together in one script. For this part, you have the process below 

## Getting Business IDs

- In order to pull the reveiws, you will need the business ids. So your first step will be to get all of the business ids from your businesses csv. 

### Open file and slice ID

1. Open data/wineries.csv
2. Slice out the 'name' and 'id' columns for each row

## Requesting Reviews

- Write a function that takes a business id and makes a call to the API for reviews.


### Adjust prior request function

Given 'id'...

GET https://api.yelp.com/v3/businesses/{id}/reviews

In [ ]:
# def yelp_request(business_id, yelp_key, verbose=True):
#     '''Adapted from Yelp API Lab: https://github.com/BenJMcCarty/dsc-yelp-api-lab/tree/solution'''
    
#     url = 'https://api.yelp.com/v3/{id}/reviews'

#     headers = {
#             'Authorization': 'Bearer {}'.format(yelp_key),
#         }

#     url_params = {
#                     'id': business_id.replace(' ', '+')
#                 }
    
#     response = requests.get(url, headers=headers, params=url_params)
    
#     if verbose == True:
#         print(response)
#         print(type(response.text))
#         print(response.text[:1000])
        
#     return response.json()

## Parsing Reviews

- Write a function to parse out the relevant information from the reviews

### Adjust prior function

In [88]:
# def parse_data(list_of_data):
#     '''Adapted from Tyrell's code'''  


#     parsed_data = []
     
#     for business in list_of_data:
#         if 'price' in business:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'price': business['price'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
#         else:
#             details = {'name': business['name'],
#                          'location': business['location']['display_address'],
#                          'id': business['id'],
#                          #'categories': business['categories'],
#                          'alias': business['categories'][0]['alias'],
#                          'title': business['categories'][0]['title'],
#                          'rating': business['rating'],
#                          'review_count': business['review_count'],
#                          'latitude': business['coordinates']['latitude'],
#                          'longitude': business['coordinates']['longitude']
#                         }
    
#         parsed_data.append(details)
    
#     for biz in parsed_data:
#         biz['location'] = ' '.join(biz['location'])
        
#     df_parsed_data = pd.DataFrame(parsed_data)
    
# #     df_parsed_data.dropna(inplace=True)
    
#     return df_parsed_data

## Saving Parsed Data

- Write a function to save the parse data into a csv file containing all of the reviews. 

### Adjust function to save to new file

In [87]:
# def get_full_data(term, location, yelp_key, file_name = 'data/wineries.csv'):
#     '''Requests all results from Yelp API; saves as a .csv; and returns a DataFrame.'''
    
#     # Process first request to Yelp API and calculate number of pages 
#     results = yelp_request_offset(term, location, yelp_key, offset=0, verbose=False)
#     num_pages = results['total']//50+1
    
#     # Print out confirmation feedback
#     print(f'For {term} and {location}: ')
#     print(f"    Total number of results: {results['total']}.")
#     print(f'    Total number of pages: {num_pages}.')
    
#     # Create offset for further results and create empty list
#     cur = 50
#     parsed_results_dfs = []

#     # Retrieves remaining pages
#     for num in range(num_pages-1):
#         try:
#             results = yelp_request_offset(term, location, yelp_key, offset=cur, verbose=False)
#             parsed_results = parse_data(results['businesses'])
#             parsed_results_dfs.append(parsed_results)
#             cur += 50
#         except:
#             print(f'Error on page {num}.')

#     # Concatenate DataFrames and save to .csv
#     df_concat = pd.concat(parsed_results_dfs, ignore_index=True)

#     try:
#         df_concat.to_csv(file_name)
#         print(f'Saved to {file_name}.')
#     except:
#         print(f'Error, did not save.')
        
#     return df_concat

## Combining Functions

- Combine the functions above into a single script  

# Part 4 -  Using python and pandas, write code to answer the questions below. 

**Reviews**

Which are the 5 most reviewed businesses in your dataset?

What is the highest rating received in your data set and how many businesses have that rating?

What percentage of businesses have a rating greater than or  4.5?

What percentage of businesses have a rating less than 3?

---

**Pricing**

What percentage of your businesses have a price label of one dollar sign? Two dollar signs? Three dollar signs? No dollar signs?

---

**Returing Reviews**

Return the text of the reviews for the most reviewed business. 

Find the highest rated business and return text of the most recent review. If multiple business have the same rating, select the business with the most reviews. 

Find the lowest rated business and return text of the most recent review.  If multiple business have the same rating, select the business with the least reviews. 

___

# Reference help

###  Pagination

Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retriving 50 at a time. Processes such as these are often refered to as pagination.

Now that you have an initial response, you can examine the contents of the json container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. 

**Note: be mindful of the API rate limits. You can only make 5000 requests per day, and APIs can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

***Below is sample code that you can use to help you deal with the pagination parameter and bring all of the functions together.***


***Also, something might cause your code to break while it is running. You don't want to constantly repull the same data when this happens, so you should insert the data into the database as you call and parse it, not after you have all of the data***


In [ ]:
# create a variable  to keep track of which result you are in. 
cur = 0

#set up a while loop to go through and grab the result 
while cur < num and cur < 1000:
    #set the offset parameter to be where you currently are in the results 
    url_params['offset'] = cur
    #make your API call with the new offset number
    results = yelp_call(url_params, api_key)
    
    #after you get your results you can now use your function to parse those results
    parsed_results = parse_results(results)
    
    # use your function to insert your parsed results into the db
    db_insert(parsed_results)
    #increment the counter by 50 to move on to the next results
    cur += 20